# Create different baseline models for total solar production

In this notebook, a baseline model for predicting solar power production is established using historical energy output data without any context.

## Init, Load

In [ ]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from src.config import DATA_RAW_DIR, DATA_RAW_FILENAME

In [ ]:
df_raw = pd.read_csv(
    os.path.join(DATA_RAW_DIR, DATA_RAW_FILENAME),
    sep=";",
    usecols=[
        "installation",
        "timestamp",
        "sol_prod",
    ],
)

In [ ]:
df_raw["date"] = pd.to_datetime(df_raw["timestamp"]).dt.normalize().dt.tz_convert(None)

df_doy = (
    df_raw.groupby(["installation", "date"])
    .agg({"sol_prod": "sum"})
    .reset_index()
)

df_doy = df_doy.sort_values(["date"])

df_doy = df_doy.drop(["installation"], axis=1)

## Split

In [ ]:
y = df_doy["sol_prod"]
X = df_doy.drop(["sol_prod"], axis=1)

# last year as holdout
split_point = len(X) - 365
X_train, X_test = X.loc[:split_point], X.loc[split_point + 1 :]
y_train, y_test = y.loc[:split_point], y.loc[split_point + 1 :]

## Date Feature Regressor

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin


class DateFeatureRegressor(BaseEstimator, RegressorMixin):
    """A simple regressor that predicts solar production based on the mean production oneach day of the year."""

    def _make_date_features(self, X):
        ts = pd.to_datetime(X.iloc[:, 0])

        df_feat = pd.DataFrame(
            {
                "year": ts.dt.year,
                "day_of_year": ts.dt.dayofyear,
            }
        )

        return df_feat

    def fit(self, X, y):
        X = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        y = pd.Series(y) if not isinstance(y, pd.Series) else y.copy()
        y.name = "target"

        X_feat = self._make_date_features(X)
        X = pd.concat([X_feat.reset_index(drop=True), y.reset_index(drop=True)], axis=1)

        # sum up each days production
        X_sum = X.groupby(["day_of_year", "year"]).sum().reset_index()

        # calculate mean production for each day of the year
        self.means_ = (
            X_sum.sort_values(by=["day_of_year", "year"])
            .drop("year", axis=1)
            .groupby("day_of_year")
            .mean().reset_index()
        )

        # ensure we have a value for day 366 (leap years)
        # just copy the value from day 365
        if 366 not in self.means_["day_of_year"].values:
            row366 = self.means_.loc[self.means_["day_of_year"] == 365].copy()
            row366["day_of_year"] = 366
            self.means_ = pd.concat(
                [self.means_, row366], ignore_index=True
            ).sort_values("day_of_year")

        return self

    def predict(self, X):
        X = pd.DataFrame(X) if not isinstance(X, pd.DataFrame) else X.copy()
        X_feat = self._make_date_features(X)

        X = pd.merge(X_feat, self.means_, on="day_of_year", how="left")
        return X["target"].values

## Train

In [ ]:
reg = DateFeatureRegressor()

reg.fit(X_train, y_train)

## Predict

In [ ]:
y_train_pred = reg.predict(X_train)
y_pred = reg.predict(X_test)

## Plot

### Load Solstice and Equinox Dates

In [ ]:
from src.config import DATA_ORIG_DIR

solstice_equinox_df = pd.read_csv(
    os.path.join(DATA_ORIG_DIR, "solstice_equinox_berlin.csv"), sep=";"
)
solstice_equinox_df["date"] = pd.to_datetime(
    pd.to_datetime(solstice_equinox_df["date"]).dt.normalize()
)
solstice_equinox_df = solstice_equinox_df[
    solstice_equinox_df["date"].isin(X["date"])
    & (solstice_equinox_df["event"].str.contains("Solstice"))
]

### Plot Predictions

In [ ]:
series_kw = {
    "actual": {
        "marker": "",
        "linestyle": "-",
        "linewidth": 0.5,
        "alpha": 0.8,
        "color": "tab:blue",
    },
    "pred_train": {
        "marker": "",
        "linestyle": "-",
        "linewidth": 0.5,
        "alpha": 0.8,
        "color": "tab:orange",
    },
    "pred_test": {
        "marker": "",
        "linestyle": "-",
        "linewidth": 0.5,
        "alpha": 0.8,
        "color": "tab:green",
    },
}

In [ ]:
fig, axes = plt.subplots(figsize=(12, 6), nrows=3, sharex=True)
axes[0].set_title("Actual Solar Production vs Predicted")


sns.lineplot(
    data=pd.concat([X, pd.Series(y, name="sol_prod")], axis=1),
    x="date",
    y="sol_prod",
    label="Actual",
    ax=axes[0],
    **series_kw["actual"]
)
sns.lineplot(
    data=pd.concat([X_train, pd.Series(y_train_pred, name="pred")], axis=1),
    x="date",
    y="pred",
    label="Predicted (train)",
    ax=axes[0],
    **series_kw["pred_train"]
)
sns.lineplot(
    data=pd.concat(
        [X_test.reset_index(drop=True), pd.Series(y_pred, name="pred")],
        axis=1,
    ),
    x="date",
    y="pred",
    label="Predicted (test)",
    ax=axes[0],
    **series_kw["pred_test"]
)

for _, row in solstice_equinox_df.iterrows():
    axes[0].axvline(
        x=row["date"], color="gray", linestyle="--", linewidth=0.8, alpha=0.8
    )
    y_h = (
        axes[0].get_ylim()[1] * 0.9
        if row["date"].month == 12
        else axes[0].get_ylim()[1] * 0.4
    )
    axes[0].text(
        row["date"],
        y_h,
        "Solstice",
        rotation=90,
        verticalalignment="top",
        horizontalalignment="right",
        fontsize=8,
        color="gray",
    )


axes[0].set_ylabel("Solar Production (Wh)")
axes[0].legend(loc="lower left")

df_doy["pred"] = pd.concat(
    [pd.Series(y_train_pred), pd.Series(y_pred)], ignore_index=True
)

df_doy["diff"] = df_doy["pred"] - df_doy["sol_prod"]

axes[1].set_title("Difference between Predicted and Actual")
axes[1].axhline(0, color="gray", linestyle="--", linewidth=0.8, alpha=0.8)

for dates, series in zip([X_train["date"], X_test["date"]], ["train", "test"]):
    sns.lineplot(
        data=df_doy[df_doy["date"].isin(dates)],
        x="date",
        y="diff",
        label=f"{series}",
        ax=axes[1],
        **series_kw[f"pred_{series}"]
    )
axes[1].set_ylabel("Difference (Wh)")
axes[1].legend(loc="lower left")

df_doy["ratio"] = np.nan
df_doy.loc[df_doy["sol_prod"] > 0, "ratio"] = df_doy["pred"] / df_doy["sol_prod"]

axes[2].set_title("Ratio of Predicted to Actual")
axes[2].axhline(1, color="gray", linestyle="--", linewidth=0.8, alpha=0.8)
for dates, series in zip([X_train["date"], X_test["date"]], ["train", "test"]):
    sns.lineplot(
        data=df_doy[df_doy["date"].isin(dates)],
    x="date",
    y="ratio",
    label=f"{series}",
    ax=axes[2],
    **series_kw[f"pred_{series}"]
)
axes[2].set_ylabel("Ratio")
axes[2].legend(loc="lower left")
axes[2].set_ylim(0.1, 100)
axes[2].set_yscale("log")


xlabels = df_doy.loc[
    (pd.to_datetime(df_doy["date"]).dt.day == 1)
    & (pd.to_datetime(df_doy["date"]).dt.month % 3 == 0),
    "date",
]

xlabels = [(d, d.strftime("%b %Y")) for d in pd.to_datetime(xlabels)]
axes[2].set_xticks(
    [d[0] for d in sorted(xlabels)],
    labels=[d[1] for d in sorted(xlabels)],
)

plt.tight_layout()
plt.show()

## Evaluate

### Testset

In [ ]:
from src.model_evaluation.regressor_evaluation import evaluate_regressor
from datetime import datetime

results_test = evaluate_regressor(
    regressor=reg,
    y_true=y_test,
    y_pred=y_pred,
    timestamp=datetime.now(),
    model_purpose="baseline",
    special_features="reg-from-rollmean",
)

print("Evaluation Results:")
for key in [
    k
    for k in ["MAE", "MSE", "RMSE", "MAPE", "MedAE", "R2", "ExplainedVar"]
    if k in results_test
]:
    print(f"  {key}: {results_test.get(key):.4f}")

### Trainset

In [ ]:
results_train = evaluate_regressor(
    regressor=reg,
    y_true=y_train,
    y_pred=y_train_pred,
    timestamp=datetime.now(),
    model_purpose="baseline",
    special_features="reg-from-rollmean",
)

print("Evaluation Results:")
for key in [
    k
    for k in ["MAE", "MSE", "RMSE", "MAPE", "MedAE", "R2", "ExplainedVar"]
    if k in results_train
]:
    print(f"  {key}: {results_train.get(key):.4f}")

## Concluding Remarks

By using the means of the output over the last years per day, a reasonable level of accuracy in the predictions is achieved. However, there is significant room for improvement.

Future work could involve exploring more complex modeling techniques to capture the intricate relationships within the data. Additionally, incorporating more features, such as real-time weather data or satellite imagery, could further enhance model performance.